# Data loading and parsing

In this notebook we find load the data from a specific simulation and parse it to be used in the subsequent analysis notebooks.

We need to know the simulation ID from the database, the ID of the Zenodo repository where the data has been deposited, and the time zone, start and end times of the simulaton for correctly parsing the data.

We use the [pickle module](https://docs.python.org/3/library/pickle.html) to serialize the objects we create in this notebook from the parsed data and save them in binary format to easily load them back into the analysis notebooks.

In [4]:
import os
from os import path
from datetime import datetime
import pickle
import pytz
import requests

import pandas as pd
import numpy as np

import networkx as nx

from scipy.io import mmwrite

# Utility code is saved in separate python files to shorten the notebook
from utils.parsing import get_contact_list, get_infection_list, get_node_state, hour_rounder, create_contact_network, remove_nodes_with_less_edges
from utils.parsing import get_score_events, get_info_count, get_info_users, get_node_index

## Input data and settings

The input files from the simulation should be stored in the data folder, under a subfolder named as their corresponding ID.

In [68]:
# Dataset ID
# 165 (WKU), 195 (PSI 2025), 200 (Murdoch 2025)
dataset_id = 200

# Format of the dataset, options are the follow:
# 0 - old OO simulations
# 1 - prod OO simulations
# 2 - research OO simulations
# 3 - epigames
data_format = 2

# Time zone, starting and ending time
# sim_tz = "US/Mountain"
# time0 = 'Oct 29 2020 9:00AM'
# time1 = 'Nov 4 2020 12:00PM'

# Time zone, starting and ending time
# sim_tz = "US/Mountain"
# time0 = 'Feb 19 2021 9:00AM'
# time1 = 'Mar 1 2021 12:00PM'

# sim_tz = "Asia/Shanghai"
# time0 = 'Nov 20 2023 9:00AM'
# time1 = 'Dec 4 2023 12:00PM'

sim_tz = "Europe/London"
time0 = 'Jun 30 2025 8:30AM'
time1 = 'Jun 30 2025 5:30PM'

# sim_tz = "Australia/Perth"
# time0 = 'Oct 7 2025 2:35PM'
# time1 = 'Oct 7 2025 4:05PM'

In [69]:
# Settings for data analysis

# Discard transmissions when the infected node was already infected before
discard_reinfections = True

# Default contact time for transmissions that are missing an associated contact event
def_contact_time = 10

# Time delta for animation in seconds, needs to be small in comparison with the total length of the 
# simulation so the changes in the animation are smooth
anim_time_step_min = 5
anim_time_delta_sec = anim_time_step_min * 60

# Time delta for plots in some "natural" time step, for examply, hourly or daily
nat_time_step_min = 5
nat_time_delta_sec = nat_time_step_min * 60

min_total_contact_time = 1  # at least this total time (in minutes) over the two weeks to be defined as in contact
min_total_contact_count = 1 # nodes must have at least this number of edges with other nodes to be kept

# Print warning messages to the console when parsing data
print_data_warnings = False

In [70]:
data_folder = path.join('./data', str(dataset_id))
output_folder = path.join('./output', str(dataset_id))
if not path.exists(output_folder):
    os.makedirs(output_folder)

# https://howchoo.com/g/ywi5m2vkodk/working-with-datetime-objects-and-timezones-in-python
# https://itnext.io/working-with-timezone-and-python-using-pytz-library-4931e61e5152
timezone = pytz.timezone(sim_tz)
obs_date0 = timezone.localize(datetime.strptime(time0, '%b %d %Y %I:%M%p'))
obs_date1 = timezone.localize(datetime.strptime(time1, '%b %d %Y %I:%M%p'))

In [71]:
if obs_date0 and obs_date1:
    tmin = datetime.timestamp(obs_date0)
    tmax = datetime.timestamp(obs_date1)
else:
    tmin = min_time
    tmax = max_time

In [72]:
# frame = 0
# t = tmin
# frame = 0
# while t <= tmax:
#     t0 = t
#     t += nat_time_delta_sec
#     frame += 1
# print(frame + 1)

num_nat_intervals = int((nat_time_delta_sec + tmax - tmin) / nat_time_delta_sec + 1)
num_nat_intervals

110

In [73]:
# Load participants and histories

all_users = pd.read_csv(path.join(data_folder, "participants.csv"), low_memory=False) 
all_events = pd.read_csv(path.join(data_folder, "histories.csv"), low_memory=False)

users = all_users[all_users["sim_id"] == dataset_id]
if data_format < 2:
    users['random_id'] = users['id']
else:    
    users['random_id'] = users['random_id'].astype(str).str.zfill(4)

# Save the users to a pickle file
with open(path.join(data_folder, 'users.pickle'), 'wb') as f:
    pickle.dump(users, f)

events = all_events[all_events["sim_id"] == dataset_id]
events.fillna({'contact_length':0, 'peer_id':-1}, inplace=True)
events["event_start"] = events["time"] - events["contact_length"]/1000
events["event_start"] = events["event_start"].astype(int)

p2pToSim = pd.Series(users.sim_id.values, index=users.p2p_id).to_dict()
p2pToId = pd.Series(users.id.values, index=users.p2p_id).to_dict()
idTop2p = pd.Series(users.p2p_id.values, index=users.id).to_dict()
        
user_index = {}
index_user = {}
idx = 0
for kid in idTop2p:
    user_index[kid] = idx
    index_user[idx] = kid
    idx += 1

# Round min and max times to the hour
min_time = min(events['time'])
max_time = max(events['time'])
first_date = hour_rounder(datetime.fromtimestamp(min_time, tz=timezone))
last_date = hour_rounder(datetime.fromtimestamp(max_time, tz=timezone))
min_time = datetime.timestamp(first_date)
max_time = datetime.timestamp(last_date)

print("First event:", first_date)
print("Last event :", last_date)

if time0 and time1:
    print("Start time:", datetime.strptime(time0, '%b %d %Y %I:%M%p'))
    print("End time:", datetime.strptime(time1, '%b %d %Y %I:%M%p'))

print(first_date.tzinfo)

# These should return the same value
print(len(users))
print(len(idTop2p))    
print(len(p2pToId))
print(len(user_index))

First event: 2025-10-07 07:00:00+01:00
Last event : 2025-10-07 11:00:00+01:00
Start time: 2025-06-30 08:30:00
End time: 2025-06-30 17:30:00
Europe/London
91
91
91
91


At this point, we have parsed the source simulation data and we can use it to extract any information we need from it. For example, in the cell below, we get the final states of all nodes, the list of infections (all the (infectors, infectees) pairs) and the list of all contacts during the entire simulation:

In [74]:
# Get list of infections and contacts, needed to construct the network graph
state = get_node_state(user_index, events, None, p2pToId, data_format, print_data_warnings)
infections = get_infection_list(user_index, events, discard_reinfections, anim_time_delta_sec, p2pToId, data_format, print_data_warnings)
contacts = get_contact_list(user_index, events, infections, def_contact_time, p2pToId, data_format, print_data_warnings)

With the contacts and state information, we can build the network graph using the [networkx package](https://networkx.org/). The first step is to construct the full network where we remove isolated nodes:

In [75]:
G = create_contact_network(user_index, contacts, state, "final_health_state", min_total_contact_time)

print(len(G.nodes()), len(G.edges()))

removed = remove_nodes_with_less_edges(G, min_total_contact_count)

isolates = list(nx.isolates(G))
G.remove_nodes_from(isolates)

mask = users.index.isin(removed + isolates)
remids = pd.DataFrame(users[mask]['random_id'].tolist(), columns=['User ID'])
remids.to_csv(path.join(data_folder, 'removed-nodes.csv'), index=False)

print('Removed', len(remids), 'nodes without enough connections')
print('There are', len(G.nodes()), 'remaining nodes with', len(G.edges()), 'edges between them')

91 511
Removed 3 nodes without enough connections
There are 88 remaining nodes with 511 edges between them


In [76]:
# Save the full network into a pickle file for later use
with open(path.join(data_folder, 'full-network.pickle'), 'wb') as f:
    pickle.dump(G, f)

We also save the directed graph containing all the transmission trees from the simulaton:

In [77]:
# Construct a new graph using only the transmission (infection) data
T = nx.DiGraph(infections)

with open(path.join(data_folder, 'transmission-tree.pickle'), 'wb') as f:
    pickle.dump(T, f)

## Getting the largest connected subgraph

We will conduct the network analyses on the largest connected subgraph in the network, we cand find it using the code in the following cell. We don't save the subgraph yet becasue we will add some properties to the nodes later on.

In [78]:
# If the Graph has more than one component, this will return False:
print("Network is connected", nx.is_connected(G))

components = nx.connected_components(G)

subgraphs = [G.subgraph(c) for c in components]
for sg in subgraphs:
    print(len(sg.nodes()), len(sg.edges()))

# Calculate the largest connected component subgraph:
G = sorted(subgraphs, key=lambda x: len(x))[-1]

degrees = [degree for node, degree in G.degree()]

Network is connected True
88 511


## Animation of network spread on network

If we save the states of all nodes during the simulation at a give interval, we can then use those states to color the nodes in an animaton that is generated in the network properties notebook.

In [79]:
# Generate the state of all nodes in G for each frame of the animation
    
t = tmin
frame = 0
all_state = []
tstate = None
print('Calculating the state of each animation frame...')
while t <= tmax:
    t0 = t
    t += anim_time_delta_sec
    td = datetime.fromtimestamp(t, tz=timezone)
    print(frame, end=' ')
    
    # We want to include contact and infection events that either started or ended between t0 and t
    condition = ((t0 < events['event_start']) & (events['event_start'] <= t)) | ((t0 < events['time']) & (events['time'] <= t))
    tevents = events[condition]
    tstate = get_node_state(user_index, tevents, tstate, p2pToId, data_format, print_data_warnings)

    fstate = [tstate[idx] for idx in list(G.nodes())]
    all_state.append(fstate)
    frame += 1
print('\nDone')

print(f'Calculated states for {len(all_state)} frames')

Calculating the state of each animation frame...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 
Done
Calculated states for 109 frames


In [80]:
# Save the network states to a file
with open(path.join(data_folder, 'all-network-states.pickle'), 'wb') as f:
    pickle.dump(all_state, f)

In [81]:
with open(path.join(data_folder, 'network-largest_conn_comp.pickle'), 'wb') as f:
    pickle.dump(G, f)

## Contact matrices and states per "natural time interval" of the simulation

Finally, we generate and save the adjacency matrices from the contact network generated for each natural interval of the simulation (only for those nodes present in the full network), as well as the states of each node. This information will be used in the analysis notebooks for tensor factorization and risk prediction.

Also, count the "score" events from each interval to plot the behaviors such as quarantine, masking, etc.

In [82]:
# Generate the state of all nodes in G for each frame of the animation

t = tmin
frame = 0

tstate = None
print('Calculating the network for each frame of the sim...')
nodes0 = list(G.nodes()) # We only look at the nodes we already selected before (which have enough interactions over the entire period of the sim)
daily_adj = np.zeros((num_nat_intervals, len(nodes0), len(nodes0)))
daily_inf = []
daily_states = []

# Versions of daily_inf and daily_states using indexing reflecting number of nodes after filtering
daily_inf_idx0 = []
daily_states_idx0 = []
# Some notes to explain the need for these additional vars: 
# In principle, each participant gets a UUID as well as a unique numerical ID, e.g: 9157781f-a054-424b-aa23-4e85ba4e8541 (UUID) 
# and 10310 (user numerical ID). One can use the dictionaries idTop2p and p2pToId to resolve the mappings between user ID and UUIDs.
# Then, each user was assigned a numerical index from 0 to N-1, where N is the number of participants in a particular sim. 
# So for example, user with ID 10310 is 0, and user with ID 10311 is 1, and so on.
# However, as some users can be filtered out for the network analyses if they do not have any contacts, this potentially reduces the
# number of nodes in the network down to M < N. This is the size of the networkX object G. The thing is that the nodes G are labelled
# with the indices from 0 to N-1, not 0 to M-1, and as it turns out, the values in the daily_states array correspond to the later, 
# not the former, so you you would need to convert the the values in daily_inf [0 to N-1] to the [0, M-1] range. Fortunately, one can 
# do this using the nodes0 array, which is simply the list of node labels in the G object. 

while t <= tmax:
    t0 = t
    t += nat_time_delta_sec
    td = datetime.fromtimestamp(t, tz=timezone)
    print('Frame', frame+1, datetime.fromtimestamp(t0, tz=timezone).strftime('%Y-%m-%d %H:%M'), 'to', td.strftime('%Y-%m-%d %H:%M'))
    
    # We want to include contact and infection events that either started or ended between t0 and t
    condition = ((t0 < events['event_start']) & (events['event_start'] <= t)) | ((t0 < events['time']) & (events['time'] <= t))
    tevents = events[condition]
    tstate = get_node_state(user_index, tevents, tstate, p2pToId, data_format, False)
    tinf = get_infection_list(user_index, tevents, discard_reinfections, nat_time_delta_sec, p2pToId, data_format, False)
    tcontacts = get_contact_list(user_index, tevents, tinf, def_contact_time, False)
    
    tg = nx.Graph()
    tg.add_nodes_from(nodes0)
    tedges = []
    tweights = []
    if 0 < len(tcontacts):
        for p in tcontacts:
            n0 = p[0]
            n1 = p[1]
            w = tcontacts[p]            
            if n0 in nodes0 and n1 in nodes0 and 0 < w:
                tedges += [(n0, n1)]
                tweights += [w]


    daily_states.append([tstate[idx] for idx in list(tg.nodes())])    
    daily_inf.append(tinf)

    # [1,M-1] indexed variables:
    tinf_idx0 = []
    for i, j in tinf:
        tinf_idx0.append((nodes0.index(i), nodes0.index(j)))
    daily_inf_idx0.append(tinf_idx0)
    daily_states_idx0.append([daily_states[frame][nodes0.index(idx)] for idx in list(tg.nodes())])    
    
    tg.add_weighted_edges_from([(tedges[i][0], tedges[i][1], tweights[i]) for i in range(len(tedges))])
    adjm = nx.adjacency_matrix(tg).todense()
    daily_adj[frame, :, :] = adjm
    
    frame += 1
print('Done')
num_frames = frame

np.save(path.join(data_folder, 'daily-contact-matrices.npy'), daily_adj)
print(f'Saved {frame} adjacency matrices to a Pickle file.')

with open(path.join(data_folder, 'daily-transmissions.npy'), 'wb') as f:
    pickle.dump(daily_inf, f)

with open(path.join(data_folder, 'daily-transmissions-idx0.npy'), 'wb') as f:
    pickle.dump(daily_inf_idx0, f)

print(f'Saved {frame} transmissions lists to a Pickle file.')    

with open(path.join(data_folder, 'daily-node-states.pickle'), 'wb') as f:
    pickle.dump(daily_states, f)

with open(path.join(data_folder, 'daily-node-states-idx0.pickle'), 'wb') as f:
    pickle.dump(daily_states_idx0, f)

print(f'Saved {frame} states to a Pickle file.')

Calculating the network for each frame of the sim...
Frame 1 2025-06-30 08:30 to 2025-06-30 08:35
Frame 2 2025-06-30 08:35 to 2025-06-30 08:40
Frame 3 2025-06-30 08:40 to 2025-06-30 08:45
Frame 4 2025-06-30 08:45 to 2025-06-30 08:50
Frame 5 2025-06-30 08:50 to 2025-06-30 08:55
Frame 6 2025-06-30 08:55 to 2025-06-30 09:00
Frame 7 2025-06-30 09:00 to 2025-06-30 09:05
Frame 8 2025-06-30 09:05 to 2025-06-30 09:10
Frame 9 2025-06-30 09:10 to 2025-06-30 09:15
Frame 10 2025-06-30 09:15 to 2025-06-30 09:20
Frame 11 2025-06-30 09:20 to 2025-06-30 09:25
Frame 12 2025-06-30 09:25 to 2025-06-30 09:30
Frame 13 2025-06-30 09:30 to 2025-06-30 09:35
Frame 14 2025-06-30 09:35 to 2025-06-30 09:40
Frame 15 2025-06-30 09:40 to 2025-06-30 09:45
Frame 16 2025-06-30 09:45 to 2025-06-30 09:50
Frame 17 2025-06-30 09:50 to 2025-06-30 09:55
Frame 18 2025-06-30 09:55 to 2025-06-30 10:00
Frame 19 2025-06-30 10:00 to 2025-06-30 10:05
Frame 20 2025-06-30 10:05 to 2025-06-30 10:10
Frame 21 2025-06-30 10:10 to 2025-06

In [83]:
dates = []
masking = []
medication = []
quarantine_no = []
quarantine_yes = []

t = tmin
frame = 0
print('Calculating score-producing events for each frame of the sim...')
while t <= tmax:
    t0 = t
    t += nat_time_delta_sec
    td = datetime.fromtimestamp(t, tz=timezone)
    
    # We want to include events that ocurred between t0 and t
    condition = (t0 < events['time']) & (events['time'] <= t)

    tevents = events[condition]    
    score_events = get_score_events(tevents)

    date = datetime.fromtimestamp(t0, tz=timezone).strftime('%Y-%m-%d')
    mask = get_info_count(score_events, "shopMask")
    med = get_info_count(score_events, "shopMedication")
    qno = get_info_count(score_events, "noQuarantine")
    qyes = get_info_count(score_events, "quarantine")

    print(date, mask, med, qno, qyes) 

    dates.append(date)
    masking.append(mask)
    medication.append(med)
    quarantine_no.append(qno)
    quarantine_yes.append(qyes)
    
    frame += 1
print('Done')

daily_behaviors = pd.DataFrame({'date': dates, 
                                'masking': masking, 
                                'medication': medication, 
                                'quarantine_no': quarantine_no, 
                                'quarantine_yes': quarantine_yes})

with open(path.join(data_folder, 'daily-behaviors.pickle'), 'wb') as f:
    pickle.dump(daily_behaviors, f)

Calculating score-producing events for each frame of the sim...
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-06-30 0 0 0 0
2025-

## EXTRAS

## Converting data to R-compatible format

Only needed if planning to conduct difussion analysis in R.

In [65]:
print('Exporting the network for each frame of the sim...')

tstate = None

nodes0 = list(G.nodes()) # We only look at the nodes we already selected before (which have enough interactions over the entire period of the sim)
daily_adj = np.zeros((15, len(nodes0), len(nodes0)))
daily_inf = []
daily_states = []

N = len(nodes0)
toa = np.array([10000] * N)
qyes_list = []
qno_list = []
mask_list = []
med_list = []

t = tmin
frame = 0
while t <= tmax:
    t0 = t
    t += nat_time_delta_sec
    td = datetime.fromtimestamp(t, tz=timezone)
    print('Frame', frame+1, datetime.fromtimestamp(t0, tz=timezone).strftime('%Y-%m-%d %H:%M'), 'to', td.strftime('%Y-%m-%d %H:%M'))
    
    # We want to include contact and infection events that either started or ended between t0 and t
    condition = ((t0 < events['event_start']) & (events['event_start'] <= t)) | ((t0 < events['time']) & (events['time'] <= t))
    tevents = events[condition]
    tstate = get_node_state(user_index, tevents, tstate, p2pToId, data_format, False)
    tinf = get_infection_list(user_index, tevents, discard_reinfections, nat_time_delta_sec, p2pToId, data_format, False)
    tcontacts = get_contact_list(user_index, tevents, tinf, def_contact_time, p2pToId, data_format, False)
    score_events = get_score_events(tevents)
    
    tg = nx.Graph()
    tg.add_nodes_from(nodes0)
    tedges = []
    tweights = []
    if 0 < len(tcontacts):
        for p in tcontacts:
            n0 = p[0]
            n1 = p[1]
            w = tcontacts[p]            
            if n0 in nodes0 and n1 in nodes0 and 0 < w:
                tedges += [(n0, n1)]
                tweights += [w]

    tg.add_weighted_edges_from([(tedges[i][0], tedges[i][1], tweights[i]) for i in range(len(tedges))])
    m = nx.to_scipy_sparse_array(tg, format="csr")
    mmwrite(f"{data_folder}/network_{frame+1}.mtx", m)
    
    fstate = np.array([tstate[idx] for idx in list(tg.nodes())])
    inf_id = np.where((fstate == 1) | (fstate == 2))[0]
    mask = frame + 1 < toa[inf_id]
    toa[inf_id[mask]] = frame + 1

    qyes = np.zeros(N)    
    users = get_info_users(score_events, "quarantine")    
    nidx = get_node_index(users, user_index, nodes0)
    qyes[nidx] = 1
    qyes_list.append(qyes)
    
    qno = np.zeros(N)
    users = get_info_users(score_events, "noQuarantine")
    nidx = get_node_index(users, user_index, nodes0)
    qno[nidx] = 1
    qno_list.append(qno)

    mask = np.zeros(N)
    users = get_info_users(score_events, "shopMask")
    nidx = get_node_index(users, user_index, nodes0)
    mask[nidx] = 1 
    mask_list.append(mask)

    med = np.zeros(N)
    users = get_info_users(score_events, "shopMedication")
    nidx = get_node_index(users, user_index, nodes0)
    med[nidx] = 1
    med_list.append(med)
    
    frame += 1
print('Done')

Exporting the network for each frame of the sim...
Frame 1 2025-06-30 08:30 to 2025-06-30 08:35
Frame 2 2025-06-30 08:35 to 2025-06-30 08:40
Frame 3 2025-06-30 08:40 to 2025-06-30 08:45
Frame 4 2025-06-30 08:45 to 2025-06-30 08:50
Frame 5 2025-06-30 08:50 to 2025-06-30 08:55
Frame 6 2025-06-30 08:55 to 2025-06-30 09:00
Frame 7 2025-06-30 09:00 to 2025-06-30 09:05
Frame 8 2025-06-30 09:05 to 2025-06-30 09:10
Frame 9 2025-06-30 09:10 to 2025-06-30 09:15
Frame 10 2025-06-30 09:15 to 2025-06-30 09:20
Frame 11 2025-06-30 09:20 to 2025-06-30 09:25
Frame 12 2025-06-30 09:25 to 2025-06-30 09:30
Frame 13 2025-06-30 09:30 to 2025-06-30 09:35
Frame 14 2025-06-30 09:35 to 2025-06-30 09:40
Frame 15 2025-06-30 09:40 to 2025-06-30 09:45
Frame 16 2025-06-30 09:45 to 2025-06-30 09:50
Frame 17 2025-06-30 09:50 to 2025-06-30 09:55
Frame 18 2025-06-30 09:55 to 2025-06-30 10:00
Frame 19 2025-06-30 10:00 to 2025-06-30 10:05
Frame 20 2025-06-30 10:05 to 2025-06-30 10:10
Frame 21 2025-06-30 10:10 to 2025-06-3

In [66]:
qyes_all = np.concatenate(qyes_list)
qno_all = np.concatenate(qno_list)
mask_all = np.concatenate(mask_list)
med_all = np.concatenate(med_list)

toa = toa.astype(float)
toa[toa == 10000] = np.nan
toa_all = np.tile(toa, num_frames)

nodes_all = np.tile(np.arange(N) + 1, num_frames)

t_all = np.repeat(np.arange(1, num_frames + 1), N)

In [67]:
net_dat = pd.DataFrame({'idnum': nodes_all, 'qyes': qyes_all, 'qno': qno_all, 'mask': mask_all, 'med': med_all, 'tinf': toa_all, 'time': t_all})
net_dat['tinf'] = net_dat['tinf'].astype('Int64')
net_dat.to_csv(f"{data_folder}/network_data.csv", index=False)

## Exporting informaton of infected nodes

Used during PSI conference to generate input data for GLEAMviz sim.

In [ ]:
tinf_step_min = 60
tinf_delta_sec = 60 * tinf_step_min

max_frame = 4

t = tmin
frame = 0

cases = []

tstate = None
print('Calculating transmissions and case info...')
nodes0 = list(G.nodes()) # We only look at the nodes we already selected before (which have enough interactions over the entire period of the sim)
daily_adj = np.zeros((num_frames, len(nodes0), len(nodes0)))
daily_inf = []
daily_states = []
while t <= tmax and frame < max_frame:
    t0 = t
    t += tinf_delta_sec
    td = datetime.fromtimestamp(t, tz=timezone)
    print('Frame', frame+1, datetime.fromtimestamp(t0, tz=timezone).strftime('%Y-%m-%d %H:%M'), 'to', td.strftime('%Y-%m-%d %H:%M'))
    
    # We want to include contact and infection events that either started or ended between t0 and t
    condition = ((t0 < events['event_start']) & (events['event_start'] <= t)) | ((t0 < events['time']) & (events['time'] <= t))
    tevents = events[condition]
    tstate = get_node_state(user_index, tevents, tstate, False)
    tinf = get_infection_list(user_index, tevents, discard_reinfections, time_delta_sec)

    for pair in tinf:
        cases.append(index_user[pair[1]])

    frame += 1
print('Done')

countries = all_events[all_events['type'] == 'modifier']
inf_ids = []
inf_countries = []
for cid in cases:
    sel = countries[countries['user_id'] == cid]
    country = sel['modifier'].values[0].split(':')[1]
    inf_ids.append(cid)
    inf_countries.append(country)

inf_data = pd.DataFrame({
    "ID": inf_ids,
    "Country": inf_countries,
    "State": ['infectious'] * len(inf_ids)
})

print(inf_data)

inf_data.to_csv(path.join(output_folder, 'gleam-input.csv'), index=False)